<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

# Training Camp - Sept 22<sup>th</sup> 2025

***

## Notebook Title :  Select and fetch BGC-Argo data

**Author contact : [G. Maze](https://annuaire.ifremer.fr/cv/17182)**

**Disclaimer**: ⚠️ BGC-Argo support in Argopy is still in active development

**Description:**

This notebook shows how to use the [DataFetcher](https://argopy.readthedocs.io/en/v1.3.0/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher):
- to load BGC-Argo data,
- use the `params` option to restrict the list of parameters to load,
- use the `measured` option to filter parameter content.

This is basically a notebook to explore this section of the Argopy documentation: [the BGC dataset](https://argopy.readthedocs.io/en/v1.3.0/user-guide/fetching-argo-data/data_set.html#specifics-in-datafetcher).

For more details on [data selection](./select-and-fetch-data.ipynb), [data sources, user modes](./fetching-options.ipynb) and performance optimisation, please refer to the dedicated notebooks.

*This notebook was developed with Argopy version: 1.3*

***

Let's start with the usual import:

In [1]:
from argopy import DataFetcher

In [2]:
# import argopy
# argopy.set_options(erddap='https://erddap-test1.ifremer.fr/erddap')

And to prevent cell output to be too large, we won't display xarray object attributes:

In [3]:
import xarray as xr
xr.set_options(display_expand_attrs = False)

### Load data from BGC-Argo

❗Note that the [DataFetcher](https://argopy.readthedocs.io/en/v1.3.0/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher) will use the Ifremer erddap data source by default, which provides access to BGC **synthetic** profile files.

Let's try to load BGC-Argo data for the Ionian Sea in 2024:

In [4]:
box = [15, 23, 35, 39, 0, 500, '2024-01', '2025-01']
f = DataFetcher(ds='bgc').region(box)
f

<datafetcher.erddap>
⭐ Name: Ifremer erddap Argo BGC data fetcher for a space/time region
🗺  Domain: [x=15.00/23.00; y=35.00/39.00; z=0.0/500.0; t=2024-01-01/2025-01-01]
🔗 API: https://erddap.ifremer.fr/erddap
📗 Parameters: ['BBP700', 'CHLA', 'DOWNWELLING_PAR', 'DOWN_IRRADIANCE380', 'DOWN_IRRADIANCE412', 'DOWN_IRRADIANCE490', 'DOXY', 'NITRATE', 'PRES', 'PSAL', 'TEMP']
📕 BGC 'must be measured' parameters: []
🏊 User mode: standard
🟢 Dataset: bgc-s
🌥  Performances: cache=False, parallel=False

In [5]:
%%time
ds = f.data
ds.argo

CPU times: user 8.07 ms, sys: 3.72 ms, total: 11.8 ms
Wall time: 11.8 s


ErddapServerError: ''

In [ ]:
ds.data_vars

<br>

Profiles location can be map with:

In [ ]:
f.plot('trajectory');

<br>

You can also quickly look at some parameters:

In [ ]:
f.plot('DOXY');

<br>

You can check the corresponding profile index structure (pandas.dataframe):

In [ ]:
df = f.index
df.head()

#### ✏️ EXERCICE

Download BGC-Argo data for Italian BGC float 6903222 and plot oxygen values.

If necessary, you can check the [data selection](./select-and-fetch-data.ipynb) notebook for help in selecting data for a given float.

💡 Hint:
```python
    %%time
    f = DataFetcher(ds='bgc')...
    f.data

    f.plot...
```      

In [ ]:
# Your code here
f = DataFetcher(ds='bgc').float(6903222)
f.data

f.plot('DOXY')

### `params` argument

[📘 Check the dedicated documentation section here](https://argopy.readthedocs.io/en/v1.3.0/user-guide/fetching-argo-data/data_set.html#the-params-argument).

Beyond the few key BGC parameters, there exist in the Argo dataset more than 120 BGC-related variables !

Therefore, it is often the case that BGC-Argo data fetching will fail because the dataset to assemble for a given selection is too large. We recommend to look at the tutorial on [how to handle large data selection](./large-data-selection.ipynb) for simple ways to possibly go through large data requests.

**Another solution to this issue is to make explicit the list of BGC parameters you are interested in**. This is a *frugal* approach that will surely benefit the simplicity of your code and the planet by limiting data transfer to what you really need.

This can be done with the `params` argument of the [DataFetcher](https://argopy.readthedocs.io/en/v1.3.0/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher).

As an example, let's download oxygen data from the Ionian Sea in 2024.

We first create the [DataFetcher](https://argopy.readthedocs.io/en/v1.3.0/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher) instance with the appropriate options:

In [ ]:
box = [15, 23, 35, 39, 0, 500, '2024-01', '2025-01']
f = DataFetcher(ds='bgc', params='DOXY').region(box)
f

<br>

and then we trigger download and look at the data structure with the `argo` xarray accessor:

In [ ]:
%%time
ds = f.data  # Trigger data fetching and processing
ds.argo

<br>

If we check on the list of variables in this dataset, we do have only 'DOXY' for BGC variables:

In [ ]:
ds.data_vars

#### 🔍 Pro tip

There is no need to add `<PARAM>_ADUSTED` along with `<PARAM>` in the ``params`` argument. This is done automatically by Argopy, depending on the user mode chosen and the post-processing required.

For instance, in *standard* mode, `DOXY` and `DOXY_ADJUSTED` will be requested, because the final value in `DOXY` will depend on the data mode of the parameter.

In *research* mode, only `DOXY_ADJUSTED` is requested.

You can look at this page of the documentation for a [detailled description of the argopy post-processing of the Argo data as a function of the dataset and user-mode](https://argopy.readthedocs.io/en/v1.3.0/user-guide/fetching-argo-data/user_mode.html#definitions).

#### 🔍 Pro tip

In a [future release](https://github.com/euroargodev/argopy/issues/490) we will add a simple method to list all available parameters that could go into the ``params`` argument.

In the mean time, this list can be accessed using this *private* method:

In [ ]:
f.fetcher._bgc_vlist_avail

#### ✏️ EXERCICE

Load only chlorophyll data from the same regional box.

💡 Hint:
```python
%%time
ds = DataFetcher(ds='bgc', params=...
ds.argo
```

In [ ]:
# Your code here
box = [15, 23, 35, 39, 0, 500, '2024-01', '2025-01']
f = DataFetcher(ds='bgc', params='CHLA').region(box)
f.data

### 
***
Useful argopy commands:

In [ ]:
# argopy.show_versions()
# argopy.reset_options()
# argopy.show_options()
# argopy.clear_cache()
# argopy.status()

***
![logo](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/notebooks/template_argopy_training_EAONE.png)